In [ ]:
# run_ba_theta_lambda_fermi.py
# BA topology, Z in {4,16}, theta grid, lambda list, b grid
# Fermi synchronous imitation with K=0.5

import os, time
import numpy as np
import pandas as pd
import networkx as nx
from collections import deque
from numba import njit

# =====================
# CONFIG
# =====================
TOPO = "BA"
N = 500
Z_LIST = [4,16]

REPS = 100          # <-- overnight friendly; set 100 if you want
STEPS = 15000
SEED_GRAPH = 42

Lmax = 1
K_FERMI = 0.5      # <-- fixed K

B_GRID = np.round(np.linspace(1.0, 2.0, 11), 1)      # 11 points
THETA_GRID = np.round(np.linspace(0, 1.0, 11), 1)  # 11 points
LAMBDA_LIST = [0.0, 0.1, 0.2, 0.5, 0.7, 1.0, 2.0]

OUT_DIR = "."
# =====================

def build_game_graph_ba(N: int, z: int, seed: int):
    # BA uses m = z/2
    return nx.barabasi_albert_graph(N, int(z / 2), seed=seed)

def graph_to_neighbor_arrays(G: nx.Graph):
    N = G.number_of_nodes()
    degs = np.array([G.degree(i) for i in range(N)], dtype=np.int32)
    max_deg = int(degs.max())
    neigh_idx = np.empty((N, max_deg), dtype=np.int32)
    neigh_cnt = np.empty(N, dtype=np.int32)

    for i in range(N):
        nbrs = list(G.neighbors(i))
        neigh_cnt[i] = len(nbrs)
        neigh_idx[i, :len(nbrs)] = np.array(nbrs, dtype=np.int32)
        if len(nbrs) < max_deg:
            neigh_idx[i, len(nbrs):] = 0

    return neigh_idx, neigh_cnt, degs.astype(np.float64)

def build_influence_layers(G_infl: nx.Graph, Lmax: int):
    """
    inf_indices[Lmax, N, N] + inf_counts[Lmax, N]
    N=500 => OK memory-wise.
    """
    N = G_infl.number_of_nodes()
    inf_indices = np.zeros((Lmax, N, N), dtype=np.int32)
    inf_counts  = np.zeros((Lmax, N), dtype=np.int32)

    adj = [list(G_infl.neighbors(i)) for i in range(N)]

    for i in range(N):
        dist = np.full(N, -1, dtype=np.int32)
        dist[i] = 0
        q = deque([i])

        while q:
            u = q.popleft()
            du = dist[u]
            if du == Lmax:
                continue
            for v in adj[u]:
                if dist[v] == -1:
                    dist[v] = du + 1
                    q.append(v)

        for j in range(N):
            d = dist[j]
            if 0 < d <= Lmax:
                d_idx = d - 1
                c = inf_counts[d_idx, i]
                inf_indices[d_idx, i, c] = j
                inf_counts[d_idx, i] += 1

    return inf_indices, inf_counts

def alpha_from_lambda(lam: float, Lmax: int) -> np.ndarray:
    # alpha_l ∝ exp(-lam*(l-1)), l=1..Lmax => exponent uses 0..Lmax-1
    if lam == 0.0:
        a = np.ones(Lmax, dtype=np.float64)
    else:
        l = np.arange(Lmax, dtype=np.float64)
        a = np.exp(-lam * l)
    a /= a.sum()
    return a.astype(np.float64)

@njit
def run_simulation_fermi_sync(neigh_idx, neigh_cnt, degrees,
                              inf_indices, inf_counts, alpha_weights,
                              theta, b, steps, seed, K):
    N = neigh_cnt.shape[0]
    np.random.seed(seed)

    # init strategies and vigilance
    strat = np.random.randint(0, 2, N).astype(np.float64)
    vig = np.zeros(N, dtype=np.float64)
    for i in range(N):
        if strat[i] == 1.0 and np.random.random() < 0.5:
            vig[i] = 1.0

    L = alpha_weights.shape[0]
    influences = np.zeros(N, dtype=np.float64)
    payoffs = np.zeros(N, dtype=np.float64)
    new_strat = np.empty(N, dtype=np.float64)
    T_eff = np.empty(N, dtype=np.float64)

    # convergence monitoring (std of rho over window)
    win = 300
    buf = np.zeros(win, dtype=np.float64)
    buf_pos = 0
    buf_full = 0

    burnin = 800
    check_every = 50
    eps_std = 2e-4

    for t in range(steps):
        # A) Influence from vigilance on layers
        for i in range(N):
            I_i = 0.0
            for l in range(L):
                cnt = inf_counts[l, i]
                if cnt > 0:
                    s = 0.0
                    for k in range(cnt):
                        s += vig[inf_indices[l, i, k]]
                    I_i += alpha_weights[l] * (s / cnt)
            influences[i] = I_i

        # B) Vigilance (amnestic)
        for i in range(N):
            if strat[i] == 1.0:
                vig[i] = 1.0 if influences[i] >= theta else 0.0
            else:
                vig[i] = 0.0

        # C) Payoffs on GAME graph (neighbors)
        for i in range(N):
            T_eff[i] = 1.0 + (b - 1.0) * (1.0 - influences[i])

        for i in range(N):
            payoffs[i] = 0.0
            ni = neigh_cnt[i]
            for kk in range(ni):
                j = neigh_idx[i, kk]
                if strat[i] == 1.0:
                    if strat[j] == 1.0:
                        payoffs[i] += 1.0
                else:
                    if strat[j] == 1.0:
                        payoffs[i] += T_eff[i]

        # D) Imitation — synchronous Fermi with K
        for i in range(N):
            new_strat[i] = strat[i]

        for i in range(N):
            ni = neigh_cnt[i]
            if ni > 0:
                j = neigh_idx[i, np.random.randint(0, ni)]
                dp = payoffs[j] - payoffs[i]   # pi_j - pi_i

                x = dp / K
                if x > 60.0:
                    p = 1.0
                elif x < -60.0:
                    p = 0.0
                else:
                    p = 1.0 / (1.0 + np.exp(-x))

                if np.random.random() < p:
                    new_strat[i] = strat[j]

        for i in range(N):
            strat[i] = new_strat[i]

        # rho
        rho = 0.0
        for i in range(N):
            rho += strat[i]
        rho /= N

        # absorption
        if rho <= 0.0 or rho >= 1.0:
            mv = 0.0
            for i in range(N):
                mv += vig[i]
            return rho, mv / N

        # buffer
        buf[buf_pos] = rho
        buf_pos += 1
        if buf_pos == win:
            buf_pos = 0
            buf_full = 1

        # stability check
        if t > burnin and (t % check_every == 0) and buf_full == 1:
            m = 0.0
            for k in range(win):
                m += buf[k]
            m /= win
            v = 0.0
            for k in range(win):
                d = buf[k] - m
                v += d * d
            v /= win
            std = np.sqrt(v)
            if std < eps_std:
                mv = 0.0
                for i in range(N):
                    mv += vig[i]
                return m, mv / N

    # end
    if buf_full == 1:
        m = 0.0
        for k in range(win):
            m += buf[k]
        m /= win
    else:
        m = rho

    mv = 0.0
    for i in range(N):
        mv += vig[i]
    return m, mv / N

def run_reps(neigh_idx, neigh_cnt, degrees,
             inf_indices, inf_counts, alpha_weights,
             theta, b, steps, reps, base_seed, K):
    rhos = np.empty(reps, dtype=np.float64)
    vigs = np.empty(reps, dtype=np.float64)
    for r in range(reps):
        rho, v = run_simulation_fermi_sync(
            neigh_idx, neigh_cnt, degrees,
            inf_indices, inf_counts, alpha_weights,
            theta, b, steps, base_seed + r, K
        )
        rhos[r] = rho
        vigs[r] = v
    return rhos, vigs

def main():
    for Z_VAL in Z_LIST:
        out_name = os.path.join(OUT_DIR, f"DATA_THETA_LAMBDA_FERMIadd_{TOPO}_Z{Z_VAL}.csv")
        if os.path.exists(out_name):
            os.remove(out_name)

        print("\n======================================")
        print(f"RUN {TOPO} Z={Z_VAL} | N={N} | Lmax={Lmax} | K={K_FERMI} | REPS={REPS} | STEPS={STEPS}")
        print("======================================")

        # fixed graph (corr=max)
        G_game = build_game_graph_ba(N, Z_VAL, seed=SEED_GRAPH)
        neigh_idx, neigh_cnt, degrees = graph_to_neighbor_arrays(G_game)

        # influence graph = game graph (corr=max)
        inf_idx, inf_cnt = build_influence_layers(G_game, Lmax=Lmax)

        for theta in THETA_GRID:
            for lam in LAMBDA_LIST:
                alphas = alpha_from_lambda(lam, Lmax=Lmax)
                for b in B_GRID:
                    start = time.time()

                    base_seed = (10000
                                 + int(1000 * theta)
                                 + int(1000 * lam)
                                 + int(100 * b)
                                 + Z_VAL * 999)

                    rhos, vigs = run_reps(
                        neigh_idx, neigh_cnt, degrees,
                        inf_idx, inf_cnt, alphas,
                        float(theta), float(b), STEPS, REPS,
                        base_seed=base_seed,
                        K=K_FERMI
                    )

                    C_mean = float(np.mean(rhos))
                    C_std  = float(np.std(rhos))
                    V_mean = float(np.mean(vigs))
                    V_std  = float(np.std(vigs))

                    SE_C = C_std / np.sqrt(REPS)
                    SE_V = V_std / np.sqrt(REPS)

                    row = pd.DataFrame([{
                        "Topo": TOPO, "Z": Z_VAL, "Corr": "max",
                        "Lmax": Lmax, "K": K_FERMI,
                        "Theta": float(theta), "Lambda": float(lam), "b": float(b),
                        "REPS": REPS, "STEPS": STEPS,
                        "C_mean": C_mean, "C_std": C_std, "SE_C": float(SE_C),
                        "V_mean": V_mean, "V_std": V_std, "SE_V": float(SE_V),
                    }])

                    row.to_csv(out_name, mode="a", header=not os.path.exists(out_name), index=False)

                    print(f"Done: Z={Z_VAL} Th={theta:.1f} lam={lam:.1f} b={b:.1f} "
                          f"(C={C_mean:.3f}, SE={SE_C:.3g}) [{time.time()-start:.1f}s]")

if __name__ == "__main__":
    main()
